In [3]:
import cv2
import numpy as np
import argparse
import time

In [4]:
weights_dir = "D:/Github/CVPR/Finalterm/yolov3/yolov3.weights"
cfg_dir = "D:/Github/CVPR/Finalterm/yolov3/yolov3.cfg"
names_dir = "D:/Github/CVPR/Finalterm/yolov3/coco.names"

def load_yolo():
	net = cv2.dnn.readNet(weights_dir, cfg_dir)
	classes = []
	with open(names_dir, "r") as f:
		classes = [line.strip() for line in f.readlines()]

	output_layers = [layer_name for layer_name in net.getUnconnectedOutLayersNames()]
	colors = np.random.uniform(0, 255, size=(1000, 3))
	return net, classes, colors, output_layers

In [5]:
def load_image(img_path):
	img = cv2.imread(img_path)
	img = cv2.resize(img, (500, 500))
	height, width, channels = img.shape
	return img, height, width, channels

In [6]:
def detect_objects(img, net, outputLayers):			
	blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
	net.setInput(blob)
	outputs = net.forward(outputLayers)
	return blob, outputs

In [7]:
def get_box_dimensions(outputs, height, width):
	boxes = []
	confs = []
	class_ids = []
	for output in outputs:
		for detect in output:
			scores = detect[5:]
			# print(scores)
			class_id = np.argmax(scores)
			conf = scores[class_id]
			if conf > 0.2:
				center_x = int(detect[0] * width)
				center_y = int(detect[1] * height)
				w = int(detect[2] * width)
				h = int(detect[3] * height)
				x = int(center_x - w/2)
				y = int(center_y - h / 2)
				boxes.append([x, y, w, h])
				confs.append(float(conf))
				class_ids.append(class_id)
	return boxes, confs, class_ids

In [8]:
def draw_labels(boxes, confs, colors, class_ids, classes, img):
	indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
	font = cv2.FONT_HERSHEY_PLAIN
	for i in range(len(boxes)):
		if i in indexes:
			x, y, w, h = boxes[i]
			label = str(classes[class_ids[i]] + " " + str(round(confs[i], 2)))
			color = colors[i]
			cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
			cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
	cv2.imshow("Image", img)

In [9]:
def detect_img(img_path):
	model, classes, colors, output_layers = load_yolo()
	image, height, width, channels = load_image(img_path)
	blob, outputs = detect_objects(image, model, output_layers)
	boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
	draw_labels(boxes, confs, colors, class_ids, classes, image)
	while True:
		key = cv2.waitKey(1)
		if key == 27: # esc key
			break
	cv2.destroyAllWindows()
	

In [10]:
detect_img('D:/Datasets/random images/derek-lee-C031beKcwdQ.jpg')